# Churchland Widefield Data

This notebook demonstrates:
- Running a ARHMM on PCA-transformed neural data
- Neural data is transformed with PCA over each brain region

In [1]:
# Import statements
import numpy as np
import pandas as pd
import ssm
import seaborn as sns
import numpy.random as npr
import matplotlib.pyplot as plt
import numpy.linalg as la
from collections import OrderedDict
from matplotlib.font_manager import FontProperties
from matplotlib.gridspec import GridSpec
from mpl_toolkits.mplot3d import Axes3D
from scipy.io import loadmat
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from ssm import LDS
from Session import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import ssm
from ssm import HMM
from ssm.util import find_permutation

In [2]:
from matplotlib.cm import get_cmap
cmap = get_cmap('coolwarm')

## 0. Specify the mouse and session date you are looking at

In [3]:
mouse = "mSM36"
day = "05-Dec-2017"

## 1. Load data and region indices

In [6]:
bl = session.behavenet_latents
bl.shape

(549, 189, 200)

In [7]:
# Load the data into SESSION, NEUR_DATA, NEUR_DATA_EXCERPT

# Sesion, and full neural data
session = Session("vistrained", mouse, day)
neur_data = session.neural['neural']
delay_period_indices = session.get_delay_period(include_stim=True)

# Taking the 120-frame excerpt
excerpt_indices = []
for trial in range(delay_period_indices.shape[0]):
    start = delay_period_indices[trial,:][0] - 30
    end = delay_period_indices[trial,:][0] + 90
    excerpt_indices.append([start, end])
trial_choices = session.trialmarkers['ResponseSide']
neur_data_excerpt = []
for trial in range(neur_data.shape[0]):
    indices = excerpt_indices[trial]
    neur_data_excerpt.append(
        bl[trial,indices[0]:indices[1],:] #change to neur_data
        )
neur_data_excerpt = np.array(neur_data_excerpt)
neur_data_excerpt_copy = neur_data_excerpt.copy()

In [9]:
# Load the neural region indices: REG_INDXS, REG_NAMES
reg_indxs = session.neural['reg_indxs_consolidate'].item()
reg_names = session.neural['reg_indxs_consolidate'].dtype.names

## 2a. Generate PCA data

In [8]:
# Generate PC data from neural data reshaped into a matrix with
# components as the covariates
n_components = 4
(num_trials, num_bins, num_comps) = neur_data_excerpt.shape
neur_data_excerpt = neur_data_excerpt.reshape(
    (-1, num_comps)
    )
pc_data = np.zeros(
    (neur_data_excerpt.shape[0], len(reg_names)*n_components)
    )

for idx, reg in enumerate(reg_names):
    pca = PCA(n_components=n_components, whiten=True)
    reg_idxs = reg_indxs[idx] - 1
    transformed_data = pca.fit_transform(
        neur_data_excerpt[:, reg_idxs.flatten()]
        )
    if n_components == 1:
        transformed_data = np.squeeze(transformed_data)
        pc_data[:, idx] = transformed_data
    else:
        pc_data[:, idx*n_components:idx*n_components + n_components] = transformed_data
    print("Region: " + reg)
    print("PCA captured variance: " + str(np.sum(pca.explained_variance_ratio_)))
    print()

pc_data = pc_data.reshape(
    (num_trials, num_bins, len(reg_names)*n_components)
    )

NameError: name 'reg_names' is not defined

## 2b. Check predictive power of PCs

In [ ]:
def fit_LR(start_idx, window_length, data, trial_choices):
    """
    Fits a L2-regularized logistic regression model, predicting
    left/right licking choice.
    
    Args
        start_idx: index in delay period to start extracting a window
            of activity.
        window_length: size of the window of activity to extract
    """
    
    X = []
    y = []
    # Extracting training and test data
    for trial in range(trial_choices.size):
        choice = trial_choices[trial]
        if np.isnan(choice):
            continue
        activity = data[trial,start_idx:start_idx+window_length,:]
        X.append(activity.flatten())
        y.append(int(choice-1))
    X = np.array(X)
    y = np.array(y)
    
    # Training the model with cross validation
    log_reg = LogisticRegressionCV(
        Cs=5, cv=5, scoring='accuracy', max_iter=500
        )
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    log_reg.fit(X, y)
    return log_reg

In [ ]:
# Do a grid search over start index and window length
# Fit logistic regression models
window_length = 2
results = []

start_idxs = range(0, 120, 2)
for start_idx in start_idxs:
    log_reg = fit_LR(start_idx, window_length, pc_data, trial_choices)
    results.append(log_reg)

In [ ]:
# Save the logistic regression models
pickle.dump(results, open(
    "pickles/temporaldecoding_neuralpca_" + mouse + "_" + day + ".p", "wb"
    ))

In [ ]:
# Extract the results corresponding to window_size = 2
with open(
    "pickles/temporaldecoding_neuralpca_" + mouse + "_" + day + ".p", "wb"
    ) as f:
    results = pickle.load(f)

In [ ]:
# Plot decoding accuracy over time
start_frame = []
accuracy = []
plt.figure(figsize=(7,5))
for frame, log_reg in enumerate(results):
    if log_reg == None:
        continue
    start_frame.append(frame*2)
    accuracy.append(
        np.max(np.mean(log_reg.scores_[1], axis=0))
        )
plt.xticks(
    ticks=[30, 30+18, 30+33, 30+51, 30+84],
    labels=['Stim On', 'Stim Off', 'Stim On', 'Stim Off', 'Spouts In'],
    rotation=45, fontsize=14
    )
plt.axvspan(30, 30+18, alpha=0.3, color='red')
plt.axvspan(30+33, 30+51, alpha=0.3, color='red')
plt.title("Choice Decoding with BehaveNet Reconstructions")
plt.ylim((0.5,1))
sns.lineplot(x=start_frame, y=accuracy)

## 3. Set the parameters of the ARHMM

In [ ]:
T = num_bins # Number of time bins
K = 0 # Number of discrete states; a hyperparameter
N = 0 # Number of observed dimensions; set in the function
num_iters = 100
kappa = 10

In [ ]:
def fit_arhmm(K, data):
    """
    Fits and returns an ARHMM on the given data
    
    Args
        K: Number of discrete states
        Data: a (trials x bins x regions) array
    Returns
        
    """
    
    N = data.shape[2]
    data = [data[i,:,:] for i in np.arange(data.shape[0])]
    
    # shuffle the trials
    shuff_sequence = [i for i in range(len(data))]
    npr.shuffle(shuff_sequence)
    shuff_sequence = np.array(shuff_sequence)
    
    # Divide into training and testing
    divider_idx = int(np.ceil(0.8*len(data)))
    train_indices = shuff_sequence[:divider_idx]
    test_indices = shuff_sequence[divider_idx:]
    data_train = [data[i] for i in train_indices]
    data_test = [data[i] for i in test_indices]
    
    # Run the ARHMM
    arhmm = HMM(
        K, N, observations="ar",
        transitions="sticky",
        transition_kwargs=dict(kappa=kappa)
        )
    lls = arhmm.fit(
        data_train, method = "em",
        num_em_iters = num_iters
        )
    
    return arhmm, lls, train_indices, test_indices

In [ ]:
K_range = np.arange(2,16,1)
K_range = np.array([2,3,4,8,12,16])

In [ ]:
results = {}
results['arhmm'] = []
results['lls'] = []
results['train_indices'] = []
results['test_indices'] = []
for K in K_range:
    arhmm, lls, train_indices, test_indices = fit_arhmm(K, pc_data)
    results['arhmm'].append(arhmm)
    results['lls'].append(lls)
    results['train_indices'].append(train_indices)
    results['test_indices'].append(test_indices)

In [ ]:
with open("pickles/arhmm_neural_" + mouse + "_" + day + ".p", "wb") as f:
    pickle.dump(results, f)

## 4. Choose the best $K$ by checking the log-likelihood

In [ ]:
with open("pickles/arhmm_neural_" + mouse + "_" + day + ".p", "rb") as f:
    results = pickle.load(f)

In [ ]:
# Plot log likelihoods if you want to see convergence
plt.figure(figsize=(12, 8))
all_lls = []
for idx, K in enumerate(K_range):
    lls = results['lls'][idx]
    line = None
    color = line.get_color() if line is not None else None
    line = plt.plot(lls, lw=1, color=color, label="K=%d"%K)[0]

xlim = plt.xlim()
plt.xlim(xlim)
plt.legend(loc="lower right")
plt.tight_layout()

In [ ]:
# Compare log likelihoods as K increases
plt.figure(figsize=(7,5))
all_lls = []
for idx, K in enumerate(K_range):
    lls = results['lls'][idx]
    all_lls.append(np.max(lls))
plt.plot(K_range, all_lls)
plt.xlabel("Number of Discrete States", fontsize=15)
plt.ylabel("Log Likelihood", fontsize=15)
plt.tight_layout()

In [ ]:
best_K = 12
best_K_idx = np.argwhere(K_range == best_K)[0,0]

## 5. Check how well the "smoothed" version looks

In [ ]:
best_arhmm = results['arhmm'][best_K_idx]
test_indices = results['test_indices'][best_K_idx]
train_indices = results['train_indices'][best_K_idx]

In [ ]:
plt.figure(figsize=(8,5))
trial = test_indices[npr.randint(test_indices.size)]
num_range = np.arange(1,15)
test = best_arhmm.smooth(pc_data[trial,:,:])
plt.plot(num_range + test[:,num_range], 'k--', label="Smoothed")
plt.plot(num_range + pc_data[trial,:,num_range].T, label="Real", color = 'r')
plt.title("Reconstructions from Trial %d"%trial)
plt.show()

## 6. Plot the inferred states

In [ ]:
trial_states = []
for trial in np.arange(pc_data.shape[0]):
    states = best_arhmm.most_likely_states(pc_data[trial,:,:])
    trial_states.append(states)
trial_states = np.array(trial_states)
plt.figure(figsize=(15,13))
plt.imshow(trial_states)
plt.xticks(
    ticks=[30, 30+18, 30+33, 30+51, 30+84],
    labels=['Stim On', 'Stim Off', 'Stim On', 'Stim Off', 'Spouts In'],
    rotation=45, fontsize=14
    )
plt.show()

## 7. Plot inferred states for L/R choices

In [ ]:
# Plot states for left or right lick choice
l_states = []
r_states = []
for trial, choice in enumerate(trial_choices):
    states = best_arhmm.most_likely_states(pc_data[trial,:,:])
    if choice == 1:
        l_states.append(states)
    else:
        r_states.append(states)
l_states = np.array(l_states)
r_states = np.array(r_states)

plt.figure(figsize=(10,7))
plt.imshow(l_states)
plt.xticks(
    ticks=[30, 30+18, 30+33, 30+51, 30+84],
    labels=['Stim On', 'Stim Off', 'Stim On', 'Stim Off', 'Spouts In'],
    rotation=45, fontsize=14
    )
plt.title("Discrete States for Left Lick Choice", fontsize=15)
plt.show()

plt.figure(figsize=(10,7))
plt.imshow(r_states)
plt.xticks(
    ticks=[30, 30+18, 30+33, 30+51, 30+84],
    labels=['Stim On', 'Stim Off', 'Stim On', 'Stim Off', 'Spouts In'],
    rotation=45, fontsize=14
    )
plt.title("Discrete States for Right Lick Choice", fontsize=15)
plt.show()

## 8. Plot inferred states for correct/incorrect trials

In [ ]:
trial_correctchoice = session.trialmarkers['CorrectSide']

In [ ]:
# Plot the states, separating by correctness of choice
correct_states = []
error_states = []
for trial, choice in enumerate(trial_choices):
    correctchoice = trial_correctchoice[trial]
    states = best_arhmm.most_likely_states(pc_data[trial,:,:])
    if choice == correctchoice:
        correct_states.append(states)
    else:
        error_states.append(states)
correct_states = np.array(correct_states)
error_states = np.array(error_states)

plt.figure(figsize=(15,12))
plt.imshow(correct_states)
plt.xticks(
    ticks=[30, 30+18, 30+33, 30+51, 30+84],
    labels=['Stim On', 'Stim Off', 'Stim On', 'Stim Off', 'Spouts In'],
    rotation=45, fontsize=14
    )
plt.title("Discrete States for Correct Trials", fontsize=15)
plt.show()

plt.figure(figsize=(10,5))
plt.imshow(error_states)
plt.xticks(
    ticks=[30, 30+18, 30+33, 30+51, 30+84],
    labels=['Stim On', 'Stim Off', 'Stim On', 'Stim Off', 'Spouts In'],
    rotation=45, fontsize=14
    )
plt.title("Discrete States for Error Trials", fontsize=15)
plt.show()